In [1]:
import numpy as np
%load_ext autoreload
%autoreload 2

In [11]:
from vitssm.data.mdsprites.generators import generate_videos
from vitssm.data.mdsprites.shapes import Circle, Rectangle, Triangle
from tqdm import tqdm

videos = []
for i, video in tqdm(enumerate(generate_videos(
    64,
    [Circle, Rectangle, Triangle],
    6,
    ["red", "blue", "green", "yellow", "purple"],
    "#000000",
    100,
))):
  videos.append(video[0])
  if i == 9999:
      break


1006it [00:43, 22.96it/s]


KeyboardInterrupt: 

In [13]:
import numpy as np
np.savez("md_videos.npz", np.array(videos))

In [6]:
from xformers.components.attention import ScaledDotProduct
from xformers.ops.fmha.attn_bias import LowerTriangularMask
import torch
from torch import nn

att = ScaledDotProduct(seq_len=30, to_seq_len=40)
q_proj = nn.Linear(128, 128)
k_proj = nn.Linear(128, 128)
v_proj = nn.Linear(128, 128)

q = torch.ones((32, 30, 128))
k = torch.ones((32, 30, 128))
v = torch.ones((32, 40, 128))
mask = LowerTriangularMask()

att(q_proj(q), k_proj(k), v_proj(v), mask=mask).shape

RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [32, 30] but got: [32, 40].

In [1]:
from vitssm.models.next_frame_prediction import LatentNextFramePrediction
import torch

model = LatentNextFramePrediction(frame_in_size=(32, 32), frame_out_size=(32, 32), patch_size=4, latent_dim=32, n_heads=2, n_frames=10)
x = torch.randn(32, 10, 32, 32, 3)
out = model(x)
print(out.shape)

/home/david/micromamba/envs/ViTSSM/lib/python3.12/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/david/micromamba/envs/ViTSSM/lib/python3.12/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
Triton is not available, some optimizations will not be enabled.
This is just a warning: triton is not available
Either FairScale or torch distributed is not available, MixtureOfExperts will not be exposed. Please install them if you would like to use MoE
/home/david/micromamba/envs/ViTSSM/lib/python3

torch.Size([320, 64, 32]) torch.Size([320, 64, 32]) torch.Size([320, 1, 32])
torch.Size([320, 64, 32]) torch.Size([320, 64, 32]) torch.Size([320, 1, 32])
torch.Size([320, 64, 32]) torch.Size([320, 64, 32]) torch.Size([320, 1, 32])
torch.Size([320, 64, 32]) torch.Size([320, 64, 32]) torch.Size([320, 1, 32])
torch.Size([32, 10, 32, 32, 3])


In [3]:
from omegaconf import OmegaConf
from pathlib import Path

base_config = OmegaConf.load(Path("run_configs") / "base_config.yml")
config = OmegaConf.load(Path("run_configs") / "config_run_1.yml")
conf = OmegaConf.merge(base_config, config)

In [1]:
from torchvision.datasets import MovingMNIST
from vitssm.data.datasets import NextFrameDataset


mnist = MovingMNIST(root="data", split="train", split_ratio=10, download=False)
data = NextFrameDataset(mnist)


(tensor([[[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
         [[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
         [[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
         ...,
 
 
         [[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],
 
 
      

In [ ]:
from vitssm.engine import build_optimizer
from addict import Dict
from torch import nn

config = Dict({
    "optimization": {
        "optimizer": {
            "name": "Adam",
            "args": {
                "lr": 0.001,
                "weight_decay": 0.0
            }
        }
    }    
})

model = nn.Linear(10, 10)
build_optimizer(model, config)

In [ ]:
from vitssm.data import build_dataset
from addict import Dict
import kappaconfig as kc
import os
from pathlib import Path

config = kc.from_file_uri(Path("run_configs") / "base_config.yml")
print(config.keys())
res = kc.DefaultResolver()
config = Dict(res.resolve(config))

dataset = build_dataset(config)